In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
##for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib 
# Import libraries
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
# Import Warnings 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
# Import tensorflow as the backend for Keras
from keras import backend as K
K.image_data_format()
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop#,adam
from keras.callbacks import TensorBoard
# Import required libraries for cnfusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import PIL
import IPython.display as ipd
import glob
import h5py
import plotly.graph_objs as go
import plotly.express as px
from PIL import Image
from tempfile import mktemp
from keras.models import load_model


In [ ]:
# read training data from CSV file / Q. 1.a
import os
train_df = pd.read_csv('../input/landmark-recognition-2020/train.csv')
print(train_df.shape)

sample_path = r"/kaggle/input/landmark-recognition-2020/sample_submission.csv"

df_test = pd.read_csv(sample_path)

test_path = r"/kaggle/input/landmark-recognition-2020/test"



In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.columns

In [ ]:
#imports for drawing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
#some statistics on the dataset
print("Some Statistics of the dataset:\n")
print(train_df['landmark_id'].describe())
print("\nThe median of the data is: ", train_df['landmark_id'].median(axis = 0))

In [ ]:
landmarks = train_df.groupby('landmark_id',as_index=False)['id'].count()\
    .sort_values('id',ascending=False).reset_index(drop=True)
landmarks.rename(columns={'id':'count'},inplace=True)

In [ ]:
print('Total Unique Landmarks = number of classes: {}'.format(train_df.landmark_id.nunique()))


In [ ]:
#1.b


landmark = train_df.landmark_id.value_counts()
landmark_df = pd.DataFrame({'landmark_id':landmark.index, 'frequency':landmark.values}).head(20)

landmark_df['landmark_id'] =   landmark_df.landmark_id.apply(lambda x: f'landmark_id_{x}')

fig = px.bar(landmark_df, x="frequency", y="landmark_id",color='landmark_id', orientation='h',
             hover_data=["landmark_id", "frequency"],
             height=1000,
             title='Number of images per landmark_id (Top 20 landmark_ids)')
fig.show()

In [ ]:
firstcnd = {len(landmarks[landmarks['count']<5])}
secondcnd = {len(landmarks[landmarks['count']<10])}
print(f"Number of classes with less than 5 images is ", firstcnd)
print(f"Number of classes with less than 10 images is ", secondcnd)
b = 17297
a = 41637
c = a - b
print(f"In between:", c)

In [ ]:
import glob
train_list = glob.glob('../input/landmark-recognition-2020/train/*/*/*/*')

In [ ]:
plt.rcParams["axes.grid"] = False
f, axarr = plt.subplots(4, 3, figsize=(10, 10))

curr_row = 0
for i in range(12):
    example = cv2.imread(train_list[i])
    example = example[:,:,::-1]
    
    col = i%4
    axarr[col, curr_row].imshow(example)
    if col == 3:
        curr_row += 1

In [ ]:
import os
import shutil

dtypes = {
        'id': 'str',
        'url': 'str',
        'landmark_id': 'uint32',
}
train_df = pd.read_csv("../input//landmark-recognition-2020/train.csv", dtype = dtypes)

In [ ]:
'''
Create the subfolders with the given labels from the data frame
'''
def create_folders():
    categories = np.unique(train_df.landmark_id.values)
    for i in categories:
        folder = r"../input/landmark-recognition-2020/train/" + str(i)
        if not os.path.exists(folder):
            print('created folder', i)
            os.mkdir(folder)
        else:
            print( str(i), ' exists!')

In [ ]:
'''
function to move the images to their corresponding folder
'''
def move_files():
    failed = 0
    for i, row in train_df.iterrows():
        filename = r"../input/landmark-recognition-2020/train/{}/{}.jpg".format(row.landmark_id, 
                                      row.id )
        oldfile = r"../input/landmark-recognition-2020/train/{}.jpg".format(row.id )
        if not os.path.exists(filename):
            try:
                os.rename(oldfile, filename)
                print('moved {}.jpg to {}'.format(row.id, row.landmark_id))
            except:
                failed +=1
        else:
            print('{}.jpg is in {}'.format(row.id, row.landmark_id))
    
    print('failed on {} files'.format(failed))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
'''
function to create the training validation folder
'''
def create_test_folder():
    failed = 0
    val_size = int(len(train_df)*0.1)
    val_folder = r"../input/train_val"
    if not os.path.exists(val_folder):
        os.mkdir(val_folder)
        
    for i, row in train_df.iloc[:val_size].iterrows():
        filename = r"../input/landmark-recognition-2020/train/{}/{}.jpg".format(row.landmark_id, 
                                      row.id )
        newFile = r"../input/landmark-recognition-2020/train_val/{}/{}.jpg".format(row.landmark_id, 
                                      row.id )
        folder = r"../input/train_val/{}".format(row.landmark_id )
        print('testing {}.jpg in {}'.format(row.id, row.landmark_id))
        if not os.path.exists(newFile):
            if not os.path.exists(folder):
                os.mkdir(folder)
                print('created folder', folder)
            try:
                shutil.copy2(filename, newFile)
                print('copied {}.jpg to train_val/{}'.format(row.id, row.landmark_id))
            except:
                failed +=1
        else:
            print('{}.jpg is in {}'.format(row.id, row.landmark_id))
    print('failed on {} files'.format(failed))

In [ ]:
##defining model 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy'])

In [ ]:
train_df['landmark_id'] = train_df.landmark_id.astype(str)
train_df["id"] = train_df.id.str[0]+"/"+train_df.id.str[1]+"/"+train_df.id.str[2]+"/"+train_df.id+".jpg"

In [ ]:
img_width = img_height = 150 
val_rate = 0.2
min_samples = 20
train_datagen_augmented = ImageDataGenerator(
        validation_split=val_rate,
        rotation_range=10,
        rescale=1. / 255,      
        shear_range=0.2,       
        zoom_range=0.2,        
        horizontal_flip=True)  

datagen = ImageDataGenerator(rescale=1. / 255)
# Hyperparameters
batch_size = 16
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 10
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen_augmented.flow_from_dataframe(
    train_df,
    directory="/kaggle/input/landmark-recognition-2020/train/",
    x_col="id",
    y_col="landmark_id",
    weight_col=None,
    target_size=(img_width, img_height),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    subset="training",
    interpolation="nearest",
    validate_filenames=False)

validation_generator = train_datagen_augmented.flow_from_dataframe(
    train_df,
    directory="/kaggle/input/landmark-recognition-2020/train/",
    x_col="id",
    y_col="landmark_id",
    weight_col=None,
    target_size=(img_width, img_height),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    subset="validation",
    interpolation="nearest",
    validate_filenames=False)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

In [ ]:
print("Evaluate on test data")

score = model.evaluate_generator(validation_generator)
print (score)


In [ ]:
submission = pd.read_csv("/kaggle/input/landmark-recognition-2020/sample_submission.csv")
submission["id"] = submission.id.str[0]+"/"+submission.id.str[1]+"/"+submission.id.str[2]+"/"+submission.id+".jpg"

trained_model = load_model(first_try.h5)
test_generator = ImageDataGenerator().flow_from_dataframe(
    submission,
    directory="/kaggle/input/landmark-recognition-2020/test/",
    x_col="id",
    y_col=None,
    weight_col=None,
    target_size=(img_width, img_height),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    batch_size=1,
    shuffle=False,
    subset=None,
    interpolation="nearest",
    validate_filenames=False)

filenames = test_generator.filenames
nb_samples = len(submission)

In [ ]:
predictions = trained_model.predict_generator(test_generator, steps = nb_samples, verbose = 1)

In [ ]:
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames

In [ ]:
# Data frame
results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

In [ ]:
### Submission
def get_image_from_number(num):
    fname = df_test.loc[num,"id"]
    fname = fname + ".jpg"
    f1 = fname[0]
    f2 = fname[1]
    f3 = fname[2]
    path = os.path.join(f1,f2,f3,fname)
    im = cv2.imread(os.path.join(test_path,path))
    return im

def get_max_class(preds):
    p = preds
    confidence = np.max(p)
    cla = np.argmax(p)
    label = decode_label(cla.reshape(1,1))[0]
    
    return label, np.round(confidence,2)
    
test_samples = len(df_test)
test_df = df_test.copy()
#for sample in range(test_samples):
  #  img = get_image_from_number(sample)
    #img = image_reshape(img, (224, 224)).reshape(1, 224, 224, 3)
    
  #  result = model.predict(img)
    
  #  label, conf = get_max_class(result)
  #  test_df.at[sample, 'landmark'] = str(label) + " " + str(conf)
  #  print(label, conf)

test_df.to_csv("submission.csv", index = False, header = True)
